In [2]:
from symbolic_formfactors import symbolic_formfactor

In [3]:
from print_latex_formfactor import latex_formfactor

In [4]:
from sympy import latex, Function, simplify, factor, multiline_latex

In [5]:
# Extract the form factor 'AL' from the symbolic_formfactor dictionary
# and its PV associated functions
AL = symbolic_formfactor['ni_GL']['AL']
AR = symbolic_formfactor['ni_GL']['AR']
AL_PV_functions = AL.atoms(Function)
AR_PV_functions = AR.atoms(Function)
# Factor out the PV functions from AL
AL_PV_factored = AL.expand().collect(AL_PV_functions, simplify)
AR_PV_factored = AR.expand().collect(AR_PV_functions, simplify)

In [6]:
AL_PV_factored

I*m_{l_a}*m_{l_b}**2*m_{n_i}*(Q_{Lai}*T_{RLib} + \overline{Q_{Lbi}}*\overline{T_{RLia}})*B1_0(m_{l_a}, m_{n_i}, m_{W_1})/(16*pi**2*k_1**3*(m_{l_a}**2 - m_{l_b}**2)) + I*m_{l_a}*m_{l_b}**2*(-Q_{Lai}*\overline{Q_{Lbi}}*m_{l_a}**2 - T_{RLib}*\overline{T_{RLia}})*B1_1(m_{l_a}, m_{n_i}, m_{W_1})/(16*pi**2*k_1**3*(m_{l_a}**2 - m_{l_b}**2))

In [7]:
print(latex(AL_PV_factored))

\frac{i m_{l_a} m_{l_b}^{2} m_{n_i} \left(Q_{Lai} T_{RLib} + \overline{Q_{Lbi}} \overline{T_{RLia}}\right) B^{\left(1\right)}_0(m_{l_a}, m_{n_i}, m_{W_1})}{16 \pi^{2} k_{1}^{3} \left(m_{l_a}^{2} - m_{l_b}^{2}\right)} + \frac{i m_{l_a} m_{l_b}^{2} \left(- Q_{Lai} \overline{Q_{Lbi}} m_{l_a}^{2} - T_{RLib} \overline{T_{RLia}}\right) B^{\left(1\right)}_1(m_{l_a}, m_{n_i}, m_{W_1})}{16 \pi^{2} k_{1}^{3} \left(m_{l_a}^{2} - m_{l_b}^{2}\right)}


In [8]:
print(latex(AR_PV_factored))

\frac{i m_{l_a}^{2} m_{l_b} \left(- Q_{Lai} \overline{Q_{Lbi}} m_{l_b}^{2} - T_{RLib} \overline{T_{RLia}}\right) B^{\left(1\right)}_1(m_{l_a}, m_{n_i}, m_{W_1})}{16 \pi^{2} k_{1}^{3} \left(m_{l_a}^{2} - m_{l_b}^{2}\right)} + \frac{i m_{l_b} m_{n_i} \left(Q_{Lai} T_{RLib} m_{l_a}^{2} + \overline{Q_{Lbi}} \overline{T_{RLia}} m_{l_b}^{2}\right) B^{\left(1\right)}_0(m_{l_a}, m_{n_i}, m_{W_1})}{16 \pi^{2} k_{1}^{3} \left(m_{l_a}^{2} - m_{l_b}^{2}\right)}


\begin{align*}
\mathtt{\text{AL(niGL)}} = & \frac{\sqrt{2} i m_{l_a} m_{l_b}^{2} m_{n_i} \left(Q_{Lai} T_{RLib} + \overline{Q_{Lbi}} \overline{T_{RLia}}\right) B^{\left(1\right)}_0(m_{l_a}, m_{n_i}, m_{W_1})}{8 \pi^{2} k_{1}^{3} \left(m_{l_a}^{2} - m_{l_b}^{2}\right)} \\
& + \frac{\sqrt{2} i m_{l_a} m_{l_b}^{2} \left(- Q_{Lai} \overline{Q_{Lbi}} m_{l_a}^{2} - T_{RLib} \overline{T_{RLia}}\right) B^{\left(1\right)}_1(m_{l_a}, m_{n_i}, m_{W_1})}{8 \pi^{2} k_{1}^{3} \left(m_{l_a}^{2} - m_{l_b}^{2}\right)} 
\end{align*}

In [9]:
from sympy import Add, Mul, fraction

In [10]:
from sympy import Add, fraction


symbolic_formfactor_factored = {}
for diagram, formfactors in symbolic_formfactor.items():
    factored_formfactors = {}
    for ff_name, ff_expr in formfactors.items():
        if ff_name == 'AL':
            ff_AL_PV_functions = ff_expr.atoms(Function)
            ff_AL_factored = ff_expr.expand().collect(ff_AL_PV_functions, simplify)
            # factor a common factor if possible
            if isinstance(ff_AL_factored, Add):
                terms = ff_AL_factored.as_ordered_terms()
                if len(terms) >1:
                    numerators = []
                    denominators = []
                    for term in terms:
                        if isinstance(term, Mul):
                            numerator, denominator = fraction(term)
                            numerators.append(numerator)
                            denominators.append(denominator)
                    # All denominators are equal?
                    cond = True
                    for i in range(1, len(denominators)):
                        if denominators[i] != denominators[0]:
                            cond = False
                        else:
                            pass
                    if cond:
                        total_numerator = Add(*numerators).expand().collect(ff_AL_PV_functions, factor)
                        ff_AL_factored = total_numerator / denominators[0]
                    else:
                        # If denominators are not equal, keep the original expression
                        ff_AL_factored = ff_AL_factored
            else:
                # If ff_AL_factored is not an Add, just use the original expression
                ff_AL_factored = ff_AL_factored

            factored_formfactors[ff_name] = ff_AL_factored
        elif ff_name == 'AR':
            ff_AR_PV_functions = ff_expr.atoms(Function)
            ff_AR_factored = ff_expr.expand().collect(ff_AR_PV_functions, simplify)
            # factor a common factor if possible
            if isinstance(ff_AR_factored, Add):
                terms = ff_AR_factored.as_ordered_terms()
                if len(terms) > 1:
                    numerators = []
                    denominators = []
                    for term in terms:
                        if isinstance(term, Mul):
                            numerator, denominator = fraction(term)
                            numerators.append(numerator)
                            denominators.append(denominator)
                    # All denominators are equal?
                    cond = True
                    for i in range(1, len(denominators)):
                        if denominators[i] != denominators[0]:
                            cond = False
                        else:
                            pass
                    if cond:
                        total_numerator = Add(*numerators).expand().collect(ff_AR_PV_functions, factor)
                        ff_AR_factored = total_numerator / denominators[0]
                    else:
                        # If denominators are not equal, keep the original expression
                        ff_AR_factored = ff_AR_factored
            factored_formfactors[ff_name] = ff_AR_factored

        else:
            # For other form factors, just use the original expression
            factored_formfactors[ff_name] = ff_expr
    symbolic_formfactor_factored[diagram] = factored_formfactors


\subsubsection{Diagram niGL}

\begin{align}
A_{L}(\text{ni\_GL})&=	\frac{\sqrt{2}im_{l_{a}}m_{l_{b}}^{2}m_{n_{i}}\left(Q_{Lai}T_{RLib}+\overline{Q_{Lbi}}\overline{T_{RLia}}\right)B_{0}^{\left(1\right)}(m_{l_{a}},m_{n_{i}},m_{W_{1}})}{8\pi^{2}k_{1}^{3}\left(m_{l_{a}}^{2}-m_{l_{b}}^{2}\right)}+\frac{\sqrt{2}im_{l_{a}}m_{l_{b}}^{2}\left(-Q_{Lai}\overline{Q_{Lbi}}m_{l_{a}}^{2}-T_{RLib}\overline{T_{RLia}}\right)B_{1}^{\left(1\right)}(m_{l_{a}},m_{n_{i}},m_{W_{1}})}{8\pi^{2}k_{1}^{3}\left(m_{l_{a}}^{2}-m_{l_{b}}^{2}\right)}\\
A_{R}(\text{ni\_GL})&=	\frac{\sqrt{2}im_{l_{a}}^{2}m_{l_{b}}\left(-Q_{Lai}\overline{Q_{Lbi}}m_{l_{b}}^{2}-T_{RLib}\overline{T_{RLia}}\right)B_{1}^{\left(1\right)}(m_{l_{a}},m_{n_{i}},m_{W_{1}})}{8\pi^{2}k_{1}^{3}\left(m_{l_{a}}^{2}-m_{l_{b}}^{2}\right)}+\frac{\sqrt{2}im_{l_{b}}m_{n_{i}}\left(Q_{Lai}T_{RLib}m_{l_{a}}^{2}+\overline{Q_{Lbi}}\overline{T_{RLia}}m_{l_{b}}^{2}\right)B_{0}^{\left(1\right)}(m_{l_{a}},m_{n_{i}},m_{W_{1}})}{8\pi^{2}k_{1}^{3}\left(m_{l_{a}}^{2}-m_{l_{b}}^{2}\right)}
\end{align}

In [11]:
two_neutrinos_diagrams = [
    'GL_ninj',
    'GR_ninj',
    'HR_ninj',
    'W1_ninj',
    'W2_ninj'
]

In [12]:
for diagram, formfactors in symbolic_formfactor.items():
    if diagram in two_neutrinos_diagrams:
        print(diagram)
        display(formfactors['AL'])

GL_ninj


I*m_{l_a}*(Q_{Lai}*T_{RLjb}*\Omega_{RLij}*m_{n_i}*m_{n_j}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j}) + Q_{Lai}*T_{RLjb}*\overline{\Omega_{RLij}}*m_{W_1}**2*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j}) - Q_{Lai}*T_{RLjb}*\overline{\Omega_{RLij}}*m_{l_a}**2*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j}) + Q_{Lai}*T_{RLjb}*\overline{\Omega_{RLij}}*m_{l_b}**2*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j}) + Q_{Lai}*T_{RLjb}*\overline{\Omega_{RLij}}*B12_0(m_{H_1^0}, m_{n_i}, m_{n_j}) - Q_{Lai}*\Omega_{RLij}*\overline{Q_{Lbj}}*m_{l_b}**2*m_{n_i}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j}) - Q_{Lai}*\Omega_{RLij}*\overline{Q_{Lbj}}*m_{l_b}**2*m_{n_i}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j}) - Q_{Lai}*\overline{Q_{Lbj}}*\overline{\Omega_{RLij}}*m_{l_b}**2*m_{n_j}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j}) - T_{RLjb}*\Omega_{RLij}*\overline{T_{RLia}}*m_{n_j}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{

GR_ninj


I*m_{l_b}*(-J_{ai}*\Omega_{RLij}*\overline{J_{bj}}*m_{n_i}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j}) - J_{ai}*\Omega_{RLij}*\overline{J_{bj}}*m_{n_i}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j}) + J_{ai}*\Omega_{RLij}*\overline{Q_{Rbj}}*m_{n_i}*m_{n_j}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j}) - J_{ai}*\overline{J_{bj}}*\overline{\Omega_{RLij}}*m_{n_j}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j}) + J_{ai}*\overline{Q_{Rbj}}*\overline{\Omega_{RLij}}*m_{W_2}**2*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j}) - J_{ai}*\overline{Q_{Rbj}}*\overline{\Omega_{RLij}}*m_{l_a}**2*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j}) + J_{ai}*\overline{Q_{Rbj}}*\overline{\Omega_{RLij}}*m_{l_b}**2*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j}) + J_{ai}*\overline{Q_{Rbj}}*\overline{\Omega_{RLij}}*B12_0(m_{H_1^0}, m_{n_i}, m_{n_j}) + Q_{Rai}*\Omega_{RLij}*\overline{J_{bj}}*m_{l_a}**2*C0(m_{H_1^0}, m_{l_a}, m_{l_

HR_ninj


I*m_{l_b}*(-K_{ai}*\Omega_{RLij}*\overline{K_{bj}}*m_{n_i}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j}) - K_{ai}*\Omega_{RLij}*\overline{K_{bj}}*m_{n_i}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j}) + K_{ai}*\Omega_{RLij}*\overline{Q_{Rbj}}*m_{n_i}*m_{n_j}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j}) - K_{ai}*\overline{K_{bj}}*\overline{\Omega_{RLij}}*m_{n_j}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j}) + K_{ai}*\overline{Q_{Rbj}}*\overline{\Omega_{RLij}}*m_{H_R^+}**2*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j}) - K_{ai}*\overline{Q_{Rbj}}*\overline{\Omega_{RLij}}*m_{l_a}**2*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j}) + K_{ai}*\overline{Q_{Rbj}}*\overline{\Omega_{RLij}}*m_{l_b}**2*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j}) + K_{ai}*\overline{Q_{Rbj}}*\overline{\Omega_{RLij}}*B12_0(m_{H_1^0}, m_{n_i}, m_{n_j}) + Q_{Rai}*\Omega_{RLij}*\overline{K_{bj}}*m_{l_a}**2*C0(m_{H_1^0}

W1_ninj


I*Q_{Lai}*\overline{Q_{Lbj}}*g**2*m_{l_a}*(-\Omega_{RLij}*m_{n_i}*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j}) + \overline{\Omega_{RLij}}*m_{n_j}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j}) - \overline{\Omega_{RLij}}*m_{n_j}*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j}))/(64*pi**2*k_1)

W2_ninj


I*Q_{Rai}*\overline{Q_{Rbj}}*g**2*m_{l_b}*(\Omega_{RLij}*m_{n_j}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j}) + \overline{\Omega_{RLij}}*m_{n_i}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j}) + \overline{\Omega_{RLij}}*m_{n_i}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j}))/(64*pi**2*k_1)

In [18]:
#%pip install sympy
from sympy import Symbol

# Import what's available from formfactors_functions, with fallback to Symbol definition
try:
	from formfactors_functions import QLai, QLbjc, TRLjb, TRLiac, QRai, QRbjc, Jai, Jbjc, Kai, Kbjc
except ImportError:
	QLai = Symbol('Q_{Lai}')
	QLbjc = Symbol('Q_{Lbjc}') 
	TRLjb = Symbol('T_{RLjb}')
	TRLiac = Symbol('T_{RLiac}')
	QRai = Symbol('Q_{Rai}')
	QRbjc = Symbol('Q_{Rbjc}')
	Jai = Symbol('J_{ai}')
	Jbjc = Symbol('J_{bjc}')
	Kai = Symbol('K_{ai}')
	Kbjc = Symbol('K_{bjc}')

try:
	from formfactors_functions import OmegaRLij, OmegaRLijc, OmegaSRij, OmegaSRijc
except ImportError:
	OmegaRLij = Symbol('\\Omega_{RLij}')
	OmegaRLijc = Symbol('\\overline{\\Omega_{RLij}}')
	OmegaSRij = Symbol('\\Omega_{SRij}')
	OmegaSRijc = Symbol('\\overline{\\Omega_{SRij}}')

# Define epsilon as a symbol if it's not available from the module
try:
	from DLRSM1.potential_senjanovic_HiggsDoublets import epsilon
except ImportError:
	epsilon = Symbol('epsilon')

In [174]:
QLai

Q_{Lai}

In [19]:
symbolic_formfactor_factored_two_neutrinos = {}
for diagram, formfactors in symbolic_formfactor.items():
    factored_formfactors = {}
    if diagram in two_neutrinos_diagrams:
        for ff_name, ff_expr in formfactors.items():
            if ff_name == 'AL':
                ff_AL_PV_functions = ff_expr.atoms(Function)
                ff_AL_factored = ff_expr.expand().collect([OmegaRLij, OmegaRLijc, OmegaSRij, OmegaSRijc], lambda x:x.collect(ff_AL_PV_functions, simplify))
                display(ff_AL_factored)
                factored_formfactors[ff_name] = ff_AL_factored
            elif ff_name == 'AR':
                ff_AR_PV_functions = ff_expr.atoms(Function)
                ff_AR_factored = ff_expr.expand().collect([OmegaRLij, OmegaRLijc, OmegaSRij, OmegaSRijc], lambda x:x.collect(ff_AR_PV_functions, simplify))
                display(ff_AR_factored)
                factored_formfactors[ff_name] = ff_AR_factored
            else:
                # For other form factors, just use the original expression
                factored_formfactors[ff_name] = ff_expr
        symbolic_formfactor_factored_two_neutrinos[diagram] = factored_formfactors
            

\Omega_{RLij}*(I*\overline{Q_{Lbj}}*m_{l_a}*m_{l_b}**2*(-Q_{Lai}*m_{n_i} + \overline{T_{RLia}})*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*\overline{T_{RLia}}*m_{l_a}*(T_{RLjb}*m_{n_j} - \overline{Q_{Lbj}}*m_{l_b}**2)*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*m_{l_a}*(Q_{Lai}*T_{RLjb}*m_{n_i}*m_{n_j} - Q_{Lai}*\overline{Q_{Lbj}}*m_{l_b}**2*m_{n_i} - T_{RLjb}*\overline{T_{RLia}}*m_{n_j} + \overline{Q_{Lbj}}*\overline{T_{RLia}}*m_{l_b}**2)*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3)) + \overline{\Omega_{RLij}}*(I*Q_{Lai}*T_{RLjb}*m_{W_1}**2*m_{l_a}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*Q_{Lai}*T_{RLjb}*m_{l_a}*B12_0(m_{H_1^0}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*Q_{Lai}*m_{l_a}*m_{l_b}**2*(T_{RLjb} - \overline{Q_{Lbj}}*m_{n_j})*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*T_{RLjb}*m_{l_a}*(-Q_{Lai}

\Omega_{RLij}*(I*\overline{Q_{Lbj}}*\overline{T_{RLia}}*m_{W_1}**2*m_{l_b}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*\overline{Q_{Lbj}}*\overline{T_{RLia}}*m_{l_b}*B12_0(m_{H_1^0}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*\overline{Q_{Lbj}}*m_{l_a}**2*m_{l_b}*(Q_{Lai}*m_{n_i} - \overline{T_{RLia}})*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*\overline{T_{RLia}}*m_{l_b}*(-T_{RLjb}*m_{n_j} + \overline{Q_{Lbj}}*m_{l_b}**2)*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3)) + \overline{\Omega_{RLij}}*(I*Q_{Lai}*m_{l_a}**2*m_{l_b}*(-T_{RLjb} + \overline{Q_{Lbj}}*m_{n_j})*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*T_{RLjb}*m_{l_b}*(Q_{Lai}*m_{l_a}**2 - \overline{T_{RLia}}*m_{n_i})*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*m_{l_b}*(Q_{Lai}*T_{RLjb}*m_{l_a}**2 - Q_{Lai}*\overline{Q_{Lbj}}*m_{l_a}**2*m_{n_j} - T_{RLjb}

\Omega_{RLij}*(I*Q_{Rai}*m_{l_a}**2*m_{l_b}*(-\overline{J_{bj}} + \overline{Q_{Rbj}}*m_{n_j})*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2) + I*\overline{J_{bj}}*m_{l_b}*(-J_{ai}*m_{n_i} + Q_{Rai}*m_{l_a}**2)*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2) + I*m_{l_b}*(-J_{ai}*\overline{J_{bj}}*m_{n_i} + J_{ai}*\overline{Q_{Rbj}}*m_{n_i}*m_{n_j} + Q_{Rai}*\overline{J_{bj}}*m_{l_a}**2 - Q_{Rai}*\overline{Q_{Rbj}}*m_{l_a}**2*m_{n_j})*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2)) + \overline{\Omega_{RLij}}*(I*J_{ai}*\overline{Q_{Rbj}}*m_{W_2}**2*m_{l_b}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2) + I*J_{ai}*\overline{Q_{Rbj}}*m_{l_b}*B12_0(m_{H_1^0}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2) + I*J_{ai}*m_{l_b}*(-\overline{J_{bj}}*m_{n_j} + \overline{Q_{Rbj}}*m_{l_b}**2)*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2) + 

\Omega_{RLij}*(I*Q_{Rai}*\overline{J_{bj}}*m_{W_2}**2*m_{l_a}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2) + I*Q_{Rai}*\overline{J_{bj}}*m_{l_a}*B12_0(m_{H_1^0}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2) + I*Q_{Rai}*m_{l_a}*m_{l_b}**2*(\overline{J_{bj}} - \overline{Q_{Rbj}}*m_{n_j})*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2) + I*\overline{J_{bj}}*m_{l_a}*(J_{ai}*m_{n_i} - Q_{Rai}*m_{l_a}**2)*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2)) + \overline{\Omega_{RLij}}*(I*J_{ai}*m_{l_a}*(\overline{J_{bj}}*m_{n_j} - \overline{Q_{Rbj}}*m_{l_b}**2)*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2) + I*\overline{Q_{Rbj}}*m_{l_a}*m_{l_b}**2*(J_{ai} - Q_{Rai}*m_{n_i})*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(32*pi**2*k_1*v_R**2) + I*m_{l_a}*(-J_{ai}*\overline{J_{bj}}*m_{n_j} + J_{ai}*\overline{Q_{Rbj}}*m_{l_b}**2 + Q_{Rai}*\overline{J_{bj}}*m

\Omega_{RLij}*(I*Q_{Rai}*m_{l_a}**2*m_{l_b}*(-\overline{K_{bj}} + \overline{Q_{Rbj}}*m_{n_j})*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*\overline{K_{bj}}*m_{l_b}*(-K_{ai}*m_{n_i} + Q_{Rai}*m_{l_a}**2)*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*m_{l_b}*(-K_{ai}*\overline{K_{bj}}*m_{n_i} + K_{ai}*\overline{Q_{Rbj}}*m_{n_i}*m_{n_j} + Q_{Rai}*\overline{K_{bj}}*m_{l_a}**2 - Q_{Rai}*\overline{Q_{Rbj}}*m_{l_a}**2*m_{n_j})*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3)) + \overline{\Omega_{RLij}}*(I*K_{ai}*\overline{Q_{Rbj}}*m_{H_R^+}**2*m_{l_b}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*K_{ai}*\overline{Q_{Rbj}}*m_{l_b}*B12_0(m_{H_1^0}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*K_{ai}*m_{l_b}*(-\overline{K_{bj}}*m_{n_j} + \overline{Q_{Rbj}}*m_{l_b}**2)*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*\overline{

\Omega_{RLij}*(I*Q_{Rai}*\overline{K_{bj}}*m_{H_R^+}**2*m_{l_a}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*Q_{Rai}*\overline{K_{bj}}*m_{l_a}*B12_0(m_{H_1^0}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*Q_{Rai}*m_{l_a}*m_{l_b}**2*(\overline{K_{bj}} - \overline{Q_{Rbj}}*m_{n_j})*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*\overline{K_{bj}}*m_{l_a}*(K_{ai}*m_{n_i} - Q_{Rai}*m_{l_a}**2)*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3)) + \overline{\Omega_{RLij}}*(I*K_{ai}*m_{l_a}*(\overline{K_{bj}}*m_{n_j} - \overline{Q_{Rbj}}*m_{l_b}**2)*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*\overline{Q_{Rbj}}*m_{l_a}*m_{l_b}**2*(K_{ai} - Q_{Rai}*m_{n_i})*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{H_R^+}, m_{n_i}, m_{n_j})/(32*pi**2*k_1**3) + I*m_{l_a}*(-K_{ai}*\overline{K_{bj}}*m_{n_j} + K_{ai}*\overline{Q_{Rbj}}*m_{l_b}**2 + Q_{Rai}*\overline{K_{bj}}*m_{n_i}*m_{n_

-I*Q_{Lai}*\Omega_{RLij}*\overline{Q_{Lbj}}*g**2*m_{l_a}*m_{n_i}*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(64*pi**2*k_1) + \overline{\Omega_{RLij}}*(I*Q_{Lai}*\overline{Q_{Lbj}}*g**2*m_{l_a}*m_{n_j}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(64*pi**2*k_1) - I*Q_{Lai}*\overline{Q_{Lbj}}*g**2*m_{l_a}*m_{n_j}*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(64*pi**2*k_1))

I*Q_{Lai}*\overline{Q_{Lbj}}*\overline{\Omega_{RLij}}*g**2*m_{l_b}*m_{n_j}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(64*pi**2*k_1) + \Omega_{RLij}*(I*Q_{Lai}*\overline{Q_{Lbj}}*g**2*m_{l_b}*m_{n_i}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(64*pi**2*k_1) + I*Q_{Lai}*\overline{Q_{Lbj}}*g**2*m_{l_b}*m_{n_i}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})/(64*pi**2*k_1))

I*Q_{Rai}*\Omega_{RLij}*\overline{Q_{Rbj}}*g**2*m_{l_b}*m_{n_j}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(64*pi**2*k_1) + \overline{\Omega_{RLij}}*(I*Q_{Rai}*\overline{Q_{Rbj}}*g**2*m_{l_b}*m_{n_i}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(64*pi**2*k_1) + I*Q_{Rai}*\overline{Q_{Rbj}}*g**2*m_{l_b}*m_{n_i}*C2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(64*pi**2*k_1))

-I*Q_{Rai}*\overline{Q_{Rbj}}*\overline{\Omega_{RLij}}*g**2*m_{l_a}*m_{n_i}*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(64*pi**2*k_1) + \Omega_{RLij}*(I*Q_{Rai}*\overline{Q_{Rbj}}*g**2*m_{l_a}*m_{n_j}*C0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(64*pi**2*k_1) - I*Q_{Rai}*\overline{Q_{Rbj}}*g**2*m_{l_a}*m_{n_j}*C1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_2}, m_{n_i}, m_{n_j})/(64*pi**2*k_1))

In [20]:
from sympy import Function, sympify
AL = Function('A_L')
AR = Function('A_R')

In [21]:
field_string_name_replacements = {
    'ni': 'n_i',
    'nj': 'n_j',
    'W1p': 'W^p',
    'W2m': 'W_2^m',
    'W1m': 'W^m',
    'W2p': 'W_2^p',
    'GRp': 'G_R^p',
    'GRm': 'G_R^m',
    'GLp': 'G_L^p',
    'GLm': 'G_L^m',
    'HRp': 'H_R^p',
    'HRm': 'H_R^m',
}
field_string_name_replacements_no_charge = {
    'GL': 'G_L',
    'GR': 'G_R',
    'HR': 'H_R',
    'W1': 'W',
    'W2': r'W_2'
    }

In [22]:
def replace_overline_with_conjugate(latex_line):
    """Replace all instances of \overline{X} with X^* in LaTeX strings, handling nested braces iteratively."""
    while '\\overline{' in latex_line:
        start = latex_line.find('\\overline{')
        # Find the matching closing brace
        open_count = 1
        end = start + 10
        while end < len(latex_line) and open_count > 0:
            if latex_line[end] == '{':
                open_count += 1
            elif latex_line[end] == '}':
                open_count -= 1
            end += 1
        if open_count == 0:
            # Process the nested content
            nested_content = latex_line[start + 10:end - 1]
            replaced_content = replace_overline_with_conjugate(nested_content) + '^*'
            latex_line = latex_line[:start] + replaced_content + latex_line[end:]
        else:
            # Unmatched braces, break the loop
            break
    return latex_line

for diagram, formfactors in symbolic_formfactor_factored_two_neutrinos.items():
    # Get LaTeX lines for AL and AR form factors
    diagram_escaped = diagram.replace('_', ',').replace('&', '\\&').replace('%', '\\%')
    # Replace field string names with their LaTeX equivalents
    for old_name, new_name in field_string_name_replacements.items():
        diagram_escaped = diagram_escaped.replace(old_name, new_name)
    for old_name, new_name in field_string_name_replacements_no_charge.items():
        diagram_escaped = diagram_escaped.replace(old_name, new_name)
    # sympify the diagram string to ensure it's valid for further processing
    if 'n_j' in diagram_escaped:
        diagram_escaped = diagram_escaped.replace('n_in_j', 'n_i*n_j')
    
    left_ff_latex = multiline_latex(AL(diagram_escaped), formfactors['AL'], environment='align*', terms_per_line=1).split('\n')
    right_ff_latex = multiline_latex(AR(diagram_escaped), formfactors['AR'], environment='align*', terms_per_line=1).split('\n')

    # Replace \overline{X} with X^* using the refined iterative function
    for line in left_ff_latex + right_ff_latex:
        if line != r"\begin{align*}" and line != r"\end{align*}":
            line = replace_overline_with_conjugate(line)
            if r'\\' in line:
                print(line)
            else:
                print(line + R'\\')
    #print('\n')

A_{L}{\left(\left( G_{L}, \  n_{i} n_{j}\right) \right)} = & \Omega_{RLij} \left(\frac{i Q_{Lbj}^* m_{l_a} m_{l_b}^{2} \left(- Q_{Lai} m_{n_i} + T_{RLia}^*\right) C_2(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})}{32 \pi^{2} k_{1}^{3}} + \frac{i T_{RLia}^* m_{l_a} \left(T_{RLjb} m_{n_j} - Q_{Lbj}^* m_{l_b}^{2}\right) C_1(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})}{32 \pi^{2} k_{1}^{3}} + \frac{i m_{l_a} \left(Q_{Lai} T_{RLjb} m_{n_i} m_{n_j} - Q_{Lai} Q_{Lbj}^* m_{l_b}^{2} m_{n_i} - T_{RLjb} T_{RLia}^* m_{n_j} + Q_{Lbj}^* T_{RLia}^* m_{l_b}^{2}\right) C_0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})}{32 \pi^{2} k_{1}^{3}}\right) \\
& + \Omega_{RLij}^* \left(\frac{i Q_{Lai} T_{RLjb} m_{W_1}^{2} m_{l_a} C_0(m_{H_1^0}, m_{l_a}, m_{l_b}, m_{W_1}, m_{n_i}, m_{n_j})}{32 \pi^{2} k_{1}^{3}} + \frac{i Q_{Lai} T_{RLjb} m_{l_a} B^{\left(12\right)}_0(m_{H_1^0}, m_{n_i}, m_{n_j})}{32 \pi^{2} k_{1}^{3}} + \frac{i Q_{Lai} m_{l_a} m_{l_b}^{2} \left(T_{RLjb} - Q_{Lbj}^

<>:2: SyntaxWarning: invalid escape sequence '\o'
<>:2: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipykernel_32217/250234023.py:2: SyntaxWarning: invalid escape sequence '\o'
  """Replace all instances of \overline{X} with X^* in LaTeX strings, handling nested braces iteratively."""


In [23]:
# Open the file to write the LaTeX expressions
with open('formfactors.tex', 'w') as f:
    f.write("\\documentclass{article}\n")
    f.write("\\usepackage{amsmath}\n")
    f.write("\\begin{document}\n")
    f.write("\\section*{Form Factors for LFV Higgs Decays in the DLRSM Model}\n")
    for diagram, formfactors in symbolic_formfactor_factored.items():
        # Escape special LaTeX characters in diagram name
        #diagram_escaped = diagram.replace('_', '\\_').replace('&', '\\&').replace('%', '\\%')
        diagram_escaped = diagram.replace('_', '')
        f.write(f"\\subsubsection{{Diagram {diagram_escaped}}}\n")
        # Write Left Form Factor using multiline_latex
        left_ff_latex = multiline_latex(f'A_L(\\text{{{diagram_escaped}}})', formfactors['AL'], environment='align*')
        f.write(replace_overline_with_conjugate(left_ff_latex) + "\n")
        # Write Right Form Factor using multiline_latex
        right_ff_latex = multiline_latex(f'A_R(\\text{{{diagram_escaped}}})', formfactors['AR'], environment='align*')
        f.write(replace_overline_with_conjugate(right_ff_latex) + "\n")
    f.write("\\end{document}\n")